In [16]:
import ipywidgets as widgets
import core
import paint_link
import base64
from IPython.display import display
from ipywidgets import HTML


## RemoteQuote

In [17]:
# input_number_of_rooms = widgets.BoundedIntText(
#     value=1,
#     min=1,
#     max=30,
#     step=1,
#     description='Rooms',
#     disabled=False
# )

# display(input_number_of_rooms)

# room_button = widgets.Button(
#     description='Create Rooms',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Click me',
#     icon='check' # (FontAwesome names without the `fa-` prefix))
# )

# display(room_button)

# room_button.on_click(room_button_callback)


    

In [18]:
input_surface = widgets.Dropdown(
    options=['Wall', 'Ceiling', 'Door', 'Doorframe', 'Skirting Board', 'Elaborate Cornice', 'Window', 'Windowsill'],
    value='Wall',
    description='Surface:',
    disabled=False,
)



In [19]:
def input_surface_switch():
    
    if input_surface.value == 'Ceiling':
        surface = core.Ceiling
    elif input_surface.value == 'Door':
        surface = core.Door
    elif input_surface.value == 'Doorframe':
        surface = core.Doorframe
    elif input_surface.value == 'Skirting Board':
        surface = core.SkirtingBoard
    elif input_surface.value == 'Elaborate Cornice':
        surface = core.ElaborateCornice
    elif input_surface.value == 'Window':
        surface = core.Window
    elif input_surface.value == 'Windowsill':
        surface = core.Windowsill
    else:
        surface = core.Wall
        
    return surface


        

In [20]:
input_surface_area = widgets.BoundedFloatText(
    value=10,
    min=0,
    max=1000.0,
    step=1.0,
    description='msq:',
    disabled=False
)

# display(input_surface_area)

In [21]:
input_condition_option = widgets.Dropdown(
    options=['poor', 'okay', 'good'],
    value='good',
    description='Condition:',
    disabled=False,
)

In [22]:
# def input_condition_switch():
#     if input_condition_option.value = 'poor':
#         return 'poor'
#     elif input_condition_option.value = 'okay':
#         return 'okay'
#     else:
#         return 'good'
        
    

In [23]:
input_paint_finish_choice = widgets.ToggleButtons(
    options=['Vinyl Matt', 'Diamond Matt', 'Custom Input'],
    description='Paint Finish:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Vinyl Matt Emulsion', 'Diamond Matt Emulsion 10x Stronger', 'Input custom values for your paint'],
    value = None
#     icons=['check'] * 3
    
)




In [24]:
def togglebutton_callback(change):
    if change['new'] == 'Vinyl Matt':
        input_paint_price.value = 37.87
        input_paint_price.disabled = True
        input_paint_unit.disabled = True
        input_paint_coverage.value = 50
        input_paint_coverage.disabled = True
    elif change['new'] == 'Diamond Matt':
        input_paint_price.value = 50.03
        input_paint_price.disabled = True
        input_paint_unit.disabled = True
        input_paint_coverage.value = 50
        input_paint_coverage.disabled = True
    else:
        input_paint_price.value = 30
        input_paint_price.disabled = False
        input_paint_coverage.disabled = False
        input_paint_unit.disabled = False
        input_paint_coverage.value = 50
    
      

In [25]:
input_paint_finish_choice.observe(togglebutton_callback, 'value')

In [26]:
input_paint_price = widgets.BoundedFloatText(
    value=30,
    min=0,
    max=1000.0,
    step=1.0,
    description='£:',
    disabled=False
)

input_paint_unit = widgets.BoundedFloatText(
    value=5,
    min=0,
    max=10.0,
    step=1.0,
    description='Litres:',
    disabled=False
)

input_paint_coverage = widgets.BoundedFloatText(
    value=17,
    min=0,
    max=100.0,
    step=1.0,
    description='Coverage:',
    disabled=False
)


input_paint_info = widgets.HBox([input_paint_price, input_paint_unit, input_paint_coverage])


input_paint_info_accordion = widgets.Accordion(children=[input_paint_info], selected_index=None,)
input_paint_info_accordion.set_title(0, "Paint Information...")


input_paint_info_accordion.layout.visibility = 'visible'




In [27]:
def calculate_button_callback(*args, **kwargs):
    incorrect_inputs = False
    
    surface_class = input_surface_switch()
    
    substrate = core.PrePaintedEmulsion(condition=input_condition_option.value)
    
    surface = surface_class(input_surface_area.value, substrate=substrate)
    
    if input_paint_finish_choice.value == 'Vinyl Matt':
        applicant = paint_link.Matt()
       
    elif input_paint_finish_choice.value == 'Custom Input':
        
        applicant = core.Paint(input_paint_price.value, input_paint_unit.value, input_paint_coverage.value)
        
    elif input_paint_finish_choice.value == 'Diamond Matt':
        applicant = paint_link.Diamond()
    else:
        incorrect_inputs = True
        
        
        
    if incorrect_inputs:
        output.value = 'Incorrect Inputs'
    else:        
        paint_surface = core.PaintingSurface(surface, applicant)
    
        output.value = f'{paint_surface.get_total_price():.2f}' 
        
        VAT_price = f'{(paint_surface.get_total_price()/100) * 120 :.2f}'
        
        
        res = f'''
        
        Your RemoteQuote
        Total {paint_surface.get_total_price():.2f}
        + VAT (If applicable) {(paint_surface.get_total_price()/100) * 120 :.2f}
        '''

        #FILE
        filename = 'quote.txt'
        b64 = base64.b64encode(res.encode())
        payload = b64.decode()

        #BUTTONS
        html_buttons = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <a download="{filename}" href="data:text/csv;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info">Download Quote</button>
        </a>
        </body>
        </html>
        '''

        html_button = html_buttons.format(payload=payload,filename=filename)
        display(HTML(html_button))
        
        output.value = html_button + output.value
        
    
        
        
        

In [28]:
calculate_button = widgets.Button(
    description='Estimate Job',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Calculate',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    
    
)
calculate_button.style.button_color='palegreen'

calculate_button.on_click(calculate_button_callback)




In [29]:
output = widgets.HTML()

In [31]:


final_box = widgets.VBox([input_surface, input_surface_area, input_condition_option, input_paint_finish_choice, input_paint_info_accordion,], )

# display(calculate_button, output)

result = widgets.VBox([calculate_button, output])

tab_contents = ['Room' + str(x + 1) for x in range(30)]

children = [final_box for name in tab_contents]

tab = widgets.Tab()
tab.children = children

for i in range(len(children)):
    tab.set_title(title=tab_contents[i], index=i)

display(tab, result)

    
# def room_button_callback(*args, **kwargs):
#     tab_contents = ['Room' + str(x + 1) for x in range(input_number_of_rooms.value)]
#     children = [final_box for name in tab_contents]

#     tab = widgets.Tab()
#     tab.children = children

#     for i in range(len(children)):
#         tab.set_title(title=tab_contents[i], index=i)
#     tab
#     display(tab)

# # display(final_box)

# # tab_contents = ['Room1', 'Room2', 'Room3', 'Room4', 'Room5']








# room_button.on_click(room_button_callback)



    


Button(description='Estimate Job', icon='check', style=ButtonStyle(button_color='palegreen'), tooltip='Calcula…

HTML(value='')